In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from livelossplot import PlotLossesKeras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
import warnings
warnings.filterwarnings('ignore')
import os
from tensorflow.keras.utils import get_custom_objects

In [3]:
# set random seed globally
my_seed = 21
from numpy.random import seed
seed(my_seed)
import tensorflow as tf
tf.random.set_seed(my_seed) 
# tensorflow version
print(tf.__version__)

2.6.0


Covertype Data Set

In [4]:
# Import dataframe from https://archive.ics.uci.edu/ml/datasets/Covertype

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', 
                 names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
                            'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
                            'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
                            'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
                            'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
                            'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
                            'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
                            'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
                            'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
                            'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'class']) # class = Cover_Type

df['class'] = df['class'] - 1 #encoding classes to start from 0 and go until 6. Initial dataset had classes from 1 to 7.
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,4
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,4
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,1
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,1
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,2
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,2
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,2
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,2


In [5]:
def buildDataSet (df, test_data_fraction):
    
    encoded_df = df

    # Set the total number of classes
    nb_classes = len(encoded_df['class'].unique())

    # Creating target and features
    X = encoded_df.drop(['class'], axis=1)
    y = encoded_df['class']

    # scale the variables
    sc = StandardScaler() 
    X_scaled = sc.fit_transform(X)

    # Split into train and test set and normalize data
    X_train, X_test, y_train, y_test = train_test_split(X_scaled,y, test_size = test_data_fraction,stratify=y) #, random_state = 0)


    return encoded_df, nb_classes, X_train, X_test, y_train, y_test


class CustomWeight(tf.keras.initializers.Initializer):
    def __init__(self):
        print('gg')
    def constant_weight(self,num):
        return tf.keras.initializers.Constant(num)
    
    def normal_weight(self,mean,stddev):
        return tf.keras.initializers.RandomNormal(mean=0., stddev=1.)


def buildSequentialModel(hidden_layers_activation, first_layer_node_count, dropout_fraction, nb_classes,weight_class, wt_mean, wt_std, bias_mean, bias_std):
    # Build a Sequential Model.
    model = Sequential()
    # model.add(Flatten(input_shape=(28, 28)))
    
    model.add(Dense(first_layer_node_count, kernel_initializer=weight_class.normal_weight(wt_mean,wt_std), activation=hidden_layers_activation,bias_initializer=weight_class.normal_weight(bias_mean,bias_std)))
    model.add(Dropout(dropout_fraction))
    model.add(Dense(first_layer_node_count*0.8, kernel_initializer=weight_class.normal_weight(wt_mean,wt_std), activation=hidden_layers_activation,bias_initializer=weight_class.normal_weight(bias_mean,bias_std)))
    model.add(Dropout(dropout_fraction))
    model.add(Dense(first_layer_node_count*0.6, kernel_initializer=weight_class.normal_weight(wt_mean,wt_std), activation=hidden_layers_activation,bias_initializer=weight_class.normal_weight(bias_mean,bias_std)))
    model.add(Dropout(dropout_fraction))
    
    # Output Layer
    model.add(Dense(nb_classes, activation=hidden_layers_activation, kernel_initializer=weight_class.normal_weight(wt_mean,wt_std), bias_initializer=weight_class.normal_weight(bias_mean,bias_std)))
   
   
    return model


def Study_Activation_Functions(hidden_layers_activation, first_layer_node_count, dropout_fraction, classes, X_train, X_test, y_train, y_test,weight_class, wt_mean, wt_std, bias_mean, bias_std):
    
    # instantiate model
    model = buildSequentialModel(hidden_layers_activation, first_layer_node_count, dropout_fraction, classes,weight_class,  wt_mean, wt_std, bias_mean, bias_std)
    # compile model
    #model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=0, validation_data=(X_test, y_test))

    train_score = model.evaluate(X_train, y_train, verbose=0)[1]
    test_score = model.evaluate(X_test, y_test, verbose=0)[1] 

    predicted_classes = np.argmax(model.predict(X_test), axis=-1)
    correct_indices = np.nonzero(predicted_classes == y_test.values)[0]
    incorrect_indices = np.nonzero(predicted_classes != y_test.values)[0]

    return train_score, test_score

In [6]:
# build dataset on our dataframe
encoded_dataframe, classes, Train_FEATURE_matrix, Test_FEATURE_matrix, Train_TARGET_matrix, Test_TARGET_matrix = buildDataSet (df, 0.2)

print(f"shape of dataframe = {encoded_dataframe.shape}")
print(f"classes in dataset = {classes}")
print(f"X_train.shape = {Train_FEATURE_matrix.shape}")
print(f"X_test.shape = {Test_FEATURE_matrix.shape}")
print(f"y_train.shape = {Train_TARGET_matrix.shape}")
print(f"y_test.shape = {Test_TARGET_matrix.shape}")
print("")
print("encoded_dataframe shown below:")
encoded_dataframe

shape of dataframe = (581012, 55)
classes in dataset = 7
X_train.shape = (464809, 54)
X_test.shape = (116203, 54)
y_train.shape = (464809,)
y_test.shape = (116203,)

encoded_dataframe shown below:


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,4
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,4
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,1
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,1
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,2
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,2
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,2
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,2


In [7]:
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

get_custom_objects().update({'mish': Activation(mish)})

In [8]:
iterations = 1 # number of iterations that we plan to perform for each pair of hidden_layer_af - output_layer_af
dataframes = [pd.DataFrame() for _ in range(0, iterations)] # declare a list of empty dataframes to store the results of the iterations

for df_index in range(0, iterations):
    dataframes[df_index] = pd.DataFrame(columns = ['data_Dimension', 'Classes', 'Hidden_AF', 'TrainTest_Split', 'First_Hidden_Layer_Tensor_Count', 'Dropouts', 'Train_Accuracy', 'Test_Accuracy'])

# results_df = pd.DataFrame(columns = ['data_Dimension', 'Classes', 'Hidden_AF', 'Output_AF', 'TrainTest_Split', 'First_Hidden_Layer_Tensor_Count', 'Dropouts', 'Train_Accuracy', 'Test_Accuracy'])

Hidden_AF_list = ['mish','relu', 'gelu', 'selu', 'sigmoid', 'tanh', 'softmax']

First_Hidden_Layer_Tensor_Count = 100
TrainTest_Split = 0.25
Dropouts = 0.3
wt_mean=0
wt_std=0.5
bias_mean=0
bias_std=0.25

for df_index in range(0, iterations):
    i = 0
    for haf in range(0,len(Hidden_AF_list)):
        encoded_dataframe, classes, Train_FEATURE_matrix, Test_FEATURE_matrix, Train_TARGET_matrix, Test_TARGET_matrix = buildDataSet (df, TrainTest_Split)
        train_accuracy_score, test_accuracy_score = Study_Activation_Functions(Hidden_AF_list[haf], First_Hidden_Layer_Tensor_Count, Dropouts, classes, Train_FEATURE_matrix, Test_FEATURE_matrix, Train_TARGET_matrix, Test_TARGET_matrix,CustomWeight(), wt_mean, wt_std, bias_mean, bias_std)
        dataframes[df_index].loc[i] = [encoded_dataframe.shape, classes, Hidden_AF_list[haf], TrainTest_Split, First_Hidden_Layer_Tensor_Count, Dropouts, np.round(train_accuracy_score,3), np.round(test_accuracy_score,3)]
        i = i + 1

gg


In [9]:
results_df = dataframes[0]
results_df = results_df.rename(columns={"Train_Accuracy": "Robust_Train_Accuracy", "Test_Accuracy": "Robust_Test_Accuracy"})
for df_index in range(1, iterations):
    results_df['Robust_Train_Accuracy'] = results_df['Robust_Train_Accuracy'] + dataframes[df_index]['Train_Accuracy']
    results_df['Robust_Test_Accuracy'] = results_df['Robust_Test_Accuracy'] + dataframes[df_index]['Test_Accuracy']
results_df['Robust_Train_Accuracy'] = round((results_df['Robust_Train_Accuracy'] / iterations),4)
results_df['Robust_Test_Accuracy'] = round((results_df['Robust_Test_Accuracy'] / iterations),4)

results_df.to_csv("results_df.csv")
results_df.sort_values(by = 'Robust_Test_Accuracy', ascending=False)

,data_Dimension,Classes,Hidden_AF,TrainTest_Split,First_Hidden_Layer_Tensor_Count,Dropouts,Robust_Train_Accuracy,Robust_Test_Accuracy
0,"(581012, 55)",7,mish,0.25,100,0.3,0.365,0.365
